# Ensemble Learning

## Contents
- Bias-Variance Decomposition
    - Mathematical Definition
    - Decomposition of Expected Test Error
- Bagging and Random Forests
- Adaptive Boosting
- Gradient Boosting Machine
- XGBoost
- LightGBM

## Bias-Variance Decomposition

Bias-Variance Decomposition 은 모델의 일반화 성능을 높히기 위한 정규화 방법론들과 앙상블 방법론들의 이론적 배경이다. 실제 데이터로 학습되는 모든 모델은 어느정도의 Bias-Variance tradeoff 가 존재하기 때문에, 수리적 배경을 이해하여 모델의 성능을 높히기 위한 적절한 조치를 취해줘야 한다.

<p align="center">
    <img src="ensemble_images/bvd.png" width="400"/>
</p>

**Error Due to Bias:** 모델의 Expected prediction 과 실제 값의 차이다. 모델의 Expected prediction 이란, 같은 분포에서 나온 많은 데이터셋을 통해 많은 모델을 학습시켰을 때 해당 모델들의 평균 예측치다.

**Error Due to Variance:** 같은 분포에서 나온 많은 데이터셋을 통해 모델을 학습시켰을 때, 한 데이터포인트에 대한 전체 평균 예측치와 개별 모델의 예측치의 차이를 의미한다. 즉, 전체 모델들간의 예측치의 차이로 해석할 수 있다.

Bias 는 모델들의 전반적인 성능이 좋은지 나쁜지를 의미하고, Variance 는 각 모델들이 서로와 얼마나 다른지를 의미한다. 머신러닝의 대부분의 모델은 둘중 하나가 높다는 특징을 가지고 있기 때문에 Bias-Variance Tradeoff 라고 불린다.

특정 모델을 학습시켰을 때 test error 는 필연적으로 등장한다. Bias-Variance Decomposition 은 해당 error 가 Bias 에서 온 error 인지, variance 에서 온 error 인지, noise 에서 온 error 인지 평가해 줄 수 있다.

### Mathematical Definition

> 해당 Tutorial 의 Mathematical Definition 파트는 [Cornell CS4780 SP17](https://www.youtube.com/watch?v=zUJbRO0Wavo) Killian Weinberger 교수님의 강의를 참고하였습니다.

$P(X,Y)$ 라는 분포에서 i.i.d 로 데이터셋 $D = \{(\mathbf{x}_1, y_1), \dots, (\mathbf{x}_n,y_n)\}$ 를 추출했다고 가정하자. Regression 모델을 가정했을 때, 모든 벡터 $\mathbf{x_n}$ 에 대한 정답 라벨 $y_n$ 은 unique 하지 않고, 분포에 따라 주어진다.

> 영상에서 이 부분에 대해 갸우뚱하는 학생들을 위해 교수님이 직접 예시를 들어준다. 집에 대한 가격을 나타내는 데이터셋에 대해, 설명변수의 값이 모두 동일한 두개의 집은 무조건적으로 가격이 같지 않다.

**Expected Label:**

$$
\bar{y}(\mathbf{x}) = E_{y \vert \mathbf{x}} \left[Y\right] = \int\limits_y y \, \Pr(y \vert \mathbf{x}) \partial y
$$

i.i.d 로 추출한 수많은 데이터셋들에 대해 모델을 학습시킨 뒤, 새로운 샘플 $\mathbf{x}$ 에 대한 expected label 은 위와 같이 계산할 수 있다. 데이터셋 자체를 $P(X,Y)$ 라는 분포에서 i.i.d 추출했기 때문에 random variable 로 취급할 수 있으므로, continuous r.v. 의 Expectation 을 구하는 것과 동일하게 $y$ 값과 모델의 예측치들에 대해 $y$ 로 미분해주면 된다.

**Expected Test Error (given $h_D$):**

$$
E_{(\mathbf{x},y) \sim P} \left[ \left(h_D (\mathbf{x}) - y \right)^2 \right] = \int\limits_x \! \! \int\limits_y \left( h_D(\mathbf{x}) - y\right)^2 \Pr(\mathbf{x},y) \partial y \partial \mathbf{x}.
$$

$D$ 라는 학습 데이터를 통해 알고리즘 $\mathcal{A}$ 를 학습시켰다고 가정하자 ($h_D = \mathcal{A}(D)$).  그렇다면 해당 모델 $h_D$ 의 Expected Test Error 은 (예시가 regression 이기 때문에 squared loss 사용) 위와 같이 구할 수 있다. $D$ 는 앞서 설명했듯이 random variable 로 취급할 수 있고, $h_D$ 는 $D$ 에 대한 function 이기 때문에 마찬가지로 random variable 이다.

**Expected Classifier (given $\mathcal{A}$):**

$$
\bar{h} = E_{D \sim P^n} \left[ h_D \right] = \int\limits_D h_D \Pr(D) \partial D
$$

**$h_D$ 와 $\mathcal{A}$ 를 구분짓는게 핵심이다.** $h_D$ 는 하나의 모델이고 $\mathcal{A}$ 는 학습될 수 있는 모든 모델이다. 그렇기 때문에 위와 같이 expected classifer 은 분포 $P$ 에서 추출된 $D$ 로 학습된 모든 모델의 expectation 이라고 할 수 있다.

**Expected Test Error (given $\mathcal{A}$):**

$$
\begin{equation*}
E_{\substack{(\mathbf{x},y) \sim P\\ D \sim P^n}} \left[\left(h_{D}(\mathbf{x}) - y\right)^{2}\right] = \int_{D} \int_{\mathbf{x}} \int_{y} \left( h_{D}(\mathbf{x}) - y\right)^{2} \mathrm{P}(\mathbf{x},y) \mathrm{P}(D) \partial \mathbf{x} \partial y \partial D
\end{equation*}
$$

모든 모델에 대한 expectation 을 구했기 때문에 이제 **가능한 모든 모델에 대한 expected error 을 표현 할 수 있다.** 우리의 목적은 $P(X,Y)$ 에서 추출된 $D$ 로 학습된 모델 $\mathcal{A}$ 의 expected 성능을 구하는 것이기 때문에, 위 식이 이를 나타낸다. 이때 해당 식을 decompose 하면 정확히 어떻게 모델의 에러가 구성되어있는지 볼 수 있다.

### Decomposition of Expected Test Error (given $\mathcal{A}$)

$$
E_{\mathbf{x},y,D}\left[\left[h_{D}(\mathbf{x}) - y\right]^{2}\right]
\\
\\
= E_{\mathbf{x},y,D}\left[\left[\left(h_{D}(\mathbf{x}) - \bar{h}(\mathbf{x})\right) + \left(\bar{h}(\mathbf{x}) - y\right)\right]^{2}\right]
\\\\
= E_{\mathbf{x}, D}\left[(\bar{h}_{D}(\mathbf{x}) - \bar{h}(\mathbf{x}))^{2}\right]
\\
+2 \mathrm{\;} E_{\mathbf{x}, y, D} \left[\left(h_{D}(\mathbf{x}) - \bar{h}(\mathbf{x})\right)\left(\bar{h}(\mathbf{x}) - y\right)\right] 
\\
+E_{\mathbf{x}, y} \left[\left(\bar{h}(\mathbf{x}) - y\right)^{2}\right]
$$

첫 식은 위에서 구한 Expected Test Error 이다. 이때 expectation 안에 모델의 label 의 expectation $\bar{h}(\mathbf{x})$ 를 한번씩 빼주고 더해준다. 이를 binomial expansion 을 통해 마지막 식처럼 표현 할 수 있다. 복잡해 보이지만 결국 $(a+b)^2 = a^2+2ab+b^2$ 의 꼴이다. 이때 식의 $2ab$ 부분은 0이 된다.

$$
E_{\mathbf{x}, y, D} \left[ \left( h_{D}(\mathbf{x}) - y \right)^{2} \right] = \underbrace{E_{\mathbf{x}, D} \left[ \left(h_{D}(\mathbf{x}) - \bar{h}(\mathbf{x}) \right)^{2} \right]}_\mathrm{Variance} + E_{\mathbf{x}, y}\left[ \left( \bar{h}(\mathbf{x}) - y \right)^{2} \right]
$$

Variance 의 정의 자체가 해당 객체가 평균에서 얼마나 분산되어있는지 이기 때문에, 식의 첫번째 부분을 Variance 라고 할 수 있다. 식의 두번째 부분은 처음에 진행 한 것 처럼 binomial expansion 꼴로 표현하면 아래와 같아진다.

$$
E_{\mathbf{x}, y} \left[ \left(\bar{h}(\mathbf{x}) - y \right)^{2}\right] = E_{\mathbf{x}, y} \left[ \left(\bar{h}(\mathbf{x}) -\bar y(\mathbf{x}) )+(\bar y(\mathbf{x}) - y \right)^{2}\right]  \\\\
=\underbrace{E_{\mathbf{x}, y} \left[\left(\bar{y}(\mathbf{x}) - y\right)^{2}\right]}_\mathrm{Noise} + \underbrace{E_{\mathbf{x}} \left[\left(\bar{h}(\mathbf{x}) - \bar{y}(\mathbf{x})\right)^{2}\right]}_\mathrm{Bias^2} \\
+ 2 \mathrm{\;} E_{\mathbf{x}, y} \left[ \left(\bar{h}(\mathbf{x}) - \bar{y}(\mathbf{x})\right)\left(\bar{y}(\mathbf{x}) - y\right)\right] 
$$

동일하게 $2ab$ 부분은 사라지기 때문에, 최종적으로 expected test error 의 decomposition 은 다음과 같아진다:

$$
\underbrace{E_{\mathbf{x}, y, D} \left[\left(h_{D}(\mathbf{x}) - y\right)^{2}\right]}_\mathrm{Expected\;Test\;Error}
\\
= \underbrace{E_{\mathbf{x}, D}\left[\left(h_{D}(\mathbf{x}) - \bar{h}(\mathbf{x})\right)^{2}\right]}_\mathrm{Variance} + \underbrace{E_{\mathbf{x}, y}\left[\left(\bar{y}(\mathbf{x}) - y\right)^{2}\right]}_\mathrm{Noise} + \underbrace{E_{\mathbf{x}}\left[\left(\bar{h}(\mathbf{x}) - \bar{y}(\mathbf{x})\right)^{2}\right]}_\mathrm{Bias^2}
$$

**즉, Bias-Variance decomposition 은 한 머신러닝 알고리즘의 expected test error (혹은 성능) 은 3개의 요인으로 나누어 설명할 수 있음을 의미한다. 한개의 모델이 다른 모델들과 얼마나 다른지를 설명하는 Variance, 전체 모델들의 평균 예측치와 실제값들의 평균의 차를 나타내는 Bias (평균 성능이라고 봐도 무방하다), 그리고 설명할 수 없는 data intrinsic 한 noise 의 합으로 하나의 모델의 성능을 나타낼 수 있다.**

이상적인 알고리즘은 Bias 와 Variance 가 낮지만, 현실적으로 tradeoff 가 발생하기 때문에 필연적으로 둘중 하나는 크다. 보통 Model Compexity 가 낮은 알고리즘들 (ex. LogReg, LDA) 는 High Bias Low Variance 이며, Model Complexity 가 높은 알고리즘들 (ex. NN, Full DT, SVM) 은 Low Bias High Variance 이다.

<p align="center">
    <img src="ensemble_images/bvd2.jpg" width="400"/>
</p>

위의 그래프를 보면 Model complexity 가 높을수록 높은 Variance 를 가졌다는 의미가 와닿는다. Training sample 에 대한 에러율은 0에 수렴하지만, 같은 분포에서 추출한 또다른 데이터셋인 testing sample 에 대한 에러는 굉장히 높다. 이는 데이터셋의 작은 변화에도 모델이 민감하게 반응한다는 것을 의미하며, 자주 접하는 과적합이 해당 모델들의 고질적인 문제를 표현한다.



## Ensembles

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier

import warnings
warnings.filterwarnings('ignore')


이상적인 모델은 Bias 와 Variance 가 낮아야 하지만, 모델의 complexity 에 따라 현실적으론 Bias-Variance Tradeoff 에 의해 둘 중 하나는 필연적으로 높다.

이를 해결하기 위해 등장한 방법론들의 모음을 **Ensemble** 이라고 하는데, 이름에서 알 수 있듯이 개별적으로 어느정도의 성능을 내는 여러개의 모델을 합쳐 최종적인 성능을 끌어올리는 방법론들이다. 앙상블 방법론들은 수행하고자 하는 task 로 인해 크개 두가지 갈래로 나눌 수 있다:

- Bagging: Decreases Variance

- Boosting: Decreases Bias

## Bootstrap Aggregating: Bagging

### Bootstraps

In [4]:
x = [f'x_{i}' for i in range(1, 11)]
y = [f'y_{i}' for i in range(1, 11)]
df = pd.DataFrame({'Original': x, 'Data': y})

b1 = df.sample(frac=1, replace=True).reset_index(drop=True)
b1.columns=['Bootstrap','1']
b2 = df.sample(frac=1, replace=True).reset_index(drop=True)
b2.columns=['Bootstrap','2']
pd.concat([df, b1, b2], axis=1)

,Original,Data,Bootstrap,1,Bootstrap,2
0,x_1,y_1,x_5,y_5,x_5,y_5
1,x_2,y_2,x_5,y_5,x_5,y_5
2,x_3,y_3,x_10,y_10,x_9,y_9
3,x_4,y_4,x_1,y_1,x_4,y_4
4,x_5,y_5,x_4,y_4,x_4,y_4
5,x_6,y_6,x_8,y_8,x_7,y_7
6,x_7,y_7,x_3,y_3,x_2,y_2
7,x_8,y_8,x_1,y_1,x_9,y_9
8,x_9,y_9,x_2,y_2,x_10,y_10
9,x_10,y_10,x_1,y_1,x_3,y_3


Bagging 은 특정 모델의 분산을 낮춰주는 학습 방법론이다. 위 테이블을 예시로 기존 데이터셋은 10개의 샘플을 가지고 있다. 이때 기존 데이터셋에서 10개의 샘플을 random 으로 복원추출을 하여 $B$ 개의 새로운 데이터셋, 즉 Bootstrap 을 생성한다.

해당 Bootstrap 들로 동일한 알고리즘을 학습 한 뒤, 결과를 하나로 합치는 것이 Bagging 의 핵심이다. 이때 알고리즘은 지도학습 알고리즘이면 어떤 것을 사용해도 무방하지만, **Bagging 은 앞서 언급하였듯이 high model complexity 를 가진 알고리즘들에 효과적이다.** Bagging 자체는 알고리즘이 아니라 앙상블의 다양성을 확보하기 위한 방법론 중 하나이기 때문에, 어떤 알고리즘을 사용하냐에 따라 Bagging with Neural Networks, Bagging with SVM 등으로 불리는 명칭이 달라진다.

In [7]:
df = pd.read_csv('Bank_Personal_Loan_Modelling.csv')
X = df.drop('Personal Loan', axis=1)
y = df['Personal Loan']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
base_svm = LogisticRegression()
base_svm.fit(X_train, y_train)
base_svm.score(X_test, y_test)

0.908

In [8]:
ensemble = BaggingClassifier(base_estimator=LogisticRegression(),
            n_estimators=20,
            bootstrap=True,
            random_state=42)

ensemble.fit(X_train, y_train)
ensemble.score(X_test, y_test)


0.913

### Result Aggregating

